# 04 - ML + LLM Integration: The Optimal Architecture

## Combining Fast ML with Intelligent LLM Analysis

**Goal:** Build a tiered architecture that uses ML as a fast filter and Claude claude-opus-4-5 for deep analysis.

### The Problem with Single-Approach Solutions

| Approach | Pros | Cons |
|----------|------|------|
| **ML Only** | Fast, cheap, scalable | Misses nuanced violations |
| **LLM Only** | Understands context, reasoning | Expensive at scale, slower |

### The Optimal Architecture

```
10,000 emails/day
       │
       ▼
┌─────────────────┐
│ ML Classifier   │  ← Fast, cheap (~$0.001/email)
│ (XGBoost)       │  ← 5ms latency
└────────┬────────┘
         │
   ┌─────┴─────┐
   │           │
 8,500      1,500 
 CLEAN      flagged
              │
              ▼
┌─────────────────┐
│ Claude claude-opus-4-5    │  ← Smart, reasoning (~$0.01/email)
│ Deep Analysis   │  ← 200ms latency
└────────┬────────┘
         │
   ┌─────┴─────┐
   │           │
 1,200       300 
 Low Risk   HIGH RISK → Human Review
```

**Result:** 90% cost reduction vs all-LLM, while maintaining accuracy on hard cases.

---

In [ ]:
# Setup
from snowflake.snowpark import Session

session = Session.builder.getOrCreate()
session.use_warehouse("COMPLIANCE_DEMO_WH")
session.use_database("COMPLIANCE_DEMO")
session.use_schema("ML")

print(f"Connected as: {session.get_current_user()}")

## 1. Load ML Predictions

Start with the predictions from our XGBoost model.

In [ ]:
# Load ML predictions
print("--- ML Model Predictions Summary ---")
session.sql("""
    SELECT 
        PREDICTED_VIOLATION,
        COUNT(*) as COUNT,
        ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER (), 1) as PCT
    FROM COMPLIANCE_DEMO.ML.MODEL_PREDICTIONS_V1
    GROUP BY 1
    ORDER BY 1
""").show()

# How many did ML flag?
flagged_count = session.sql("""
    SELECT COUNT(*) FROM COMPLIANCE_DEMO.ML.MODEL_PREDICTIONS_V1 WHERE PREDICTED_VIOLATION = 1
""").collect()[0][0]
print(f"\nML flagged {flagged_count:,} emails for deeper review")

## 2. Apply Claude claude-opus-4-5 to ML-Flagged Emails

Only run the expensive LLM on emails the ML model flagged as suspicious.

In [ ]:
# Run Claude claude-opus-4-5 analysis on ML-flagged emails (sample for demo speed)
print("🤖 Running Claude claude-opus-4-5 on ML-flagged emails...")

session.sql("""
CREATE OR REPLACE TABLE COMPLIANCE_DEMO.ML.LLM_ANALYSIS AS
WITH flagged_emails AS (
    SELECT 
        p.EMAIL_ID,
        p.COMPLIANCE_LABEL as ACTUAL_LABEL,
        p.PREDICTED_VIOLATION as ML_PREDICTION,
        e.SUBJECT,
        e.BODY
    FROM COMPLIANCE_DEMO.ML.MODEL_PREDICTIONS_V1 p
    JOIN COMPLIANCE_DEMO.EMAIL_SURVEILLANCE.EMAILS e ON p.EMAIL_ID = e.EMAIL_ID
    WHERE p.PREDICTED_VIOLATION = 1
    LIMIT 50  -- Sample for demo speed
)
SELECT 
    EMAIL_ID,
    ACTUAL_LABEL,
    ML_PREDICTION,
    SUBJECT,
    
    -- Claude claude-opus-4-5 classification
    SNOWFLAKE.CORTEX.COMPLETE(
        'claude-opus-4-5',
        CONCAT(
            'You are a hedge fund compliance analyst. Analyze this email and determine if it contains a compliance violation. ',
            'Respond with ONLY a JSON object: {"is_violation": true/false, "confidence": 0.0-1.0, "category": "INSIDER_TRADING|CONFIDENTIALITY|PERSONAL_TRADING|INFO_BARRIER|CLEAN", "reasoning": "brief explanation"}. ',
            'Email subject: ', SUBJECT, ' ',
            'Email body: ', LEFT(BODY, 1500)
        )
    ) AS CLAUDE_ANALYSIS
    
FROM flagged_emails
""").collect()

print("✅ Claude analysis complete")

# Preview results
session.sql("SELECT EMAIL_ID, ACTUAL_LABEL, CLAUDE_ANALYSIS FROM COMPLIANCE_DEMO.ML.LLM_ANALYSIS LIMIT 5").show()

## 3. Compare ML vs Claude Performance

See how the two methods complement each other.

In [ ]:
# Parse Claude's JSON response and compare
print("--- ML vs Claude Comparison ---")

session.sql("""
    SELECT 
        ACTUAL_LABEL,
        ML_PREDICTION,
        TRY_PARSE_JSON(CLAUDE_ANALYSIS):is_violation::BOOLEAN as CLAUDE_VIOLATION,
        TRY_PARSE_JSON(CLAUDE_ANALYSIS):category::STRING as CLAUDE_CATEGORY,
        TRY_PARSE_JSON(CLAUDE_ANALYSIS):confidence::FLOAT as CLAUDE_CONFIDENCE,
        LEFT(TRY_PARSE_JSON(CLAUDE_ANALYSIS):reasoning::STRING, 100) as REASONING
    FROM COMPLIANCE_DEMO.ML.LLM_ANALYSIS
    LIMIT 10
""").show()

## 4. Create Ensemble Predictions

Combine ML and LLM for maximum accuracy.

In [ ]:
# Create ensemble predictions view
session.sql("""
CREATE OR REPLACE VIEW COMPLIANCE_DEMO.ML.ENSEMBLE_RESULTS AS
SELECT 
    EMAIL_ID,
    ACTUAL_LABEL,
    CASE WHEN ACTUAL_LABEL != 'CLEAN' THEN 1 ELSE 0 END as ACTUAL_VIOLATION,
    ML_PREDICTION,
    COALESCE(TRY_PARSE_JSON(CLAUDE_ANALYSIS):is_violation::BOOLEAN, FALSE) as CLAUDE_VIOLATION,
    TRY_PARSE_JSON(CLAUDE_ANALYSIS):confidence::FLOAT as CLAUDE_CONFIDENCE,
    TRY_PARSE_JSON(CLAUDE_ANALYSIS):category::STRING as CLAUDE_CATEGORY,
    
    -- Ensemble: High precision (both agree it's a violation)
    CASE WHEN ML_PREDICTION = 1 AND TRY_PARSE_JSON(CLAUDE_ANALYSIS):is_violation::BOOLEAN = TRUE 
         THEN 1 ELSE 0 END as ENSEMBLE_HIGH_PRECISION,
    
    -- Priority score for human review
    CASE 
        WHEN ML_PREDICTION = 1 AND TRY_PARSE_JSON(CLAUDE_ANALYSIS):is_violation::BOOLEAN = TRUE 
            AND TRY_PARSE_JSON(CLAUDE_ANALYSIS):confidence::FLOAT > 0.8
        THEN 'CRITICAL'
        WHEN ML_PREDICTION = 1 AND TRY_PARSE_JSON(CLAUDE_ANALYSIS):is_violation::BOOLEAN = TRUE
        THEN 'HIGH'
        WHEN ML_PREDICTION = 1
        THEN 'MEDIUM'
        ELSE 'LOW'
    END as REVIEW_PRIORITY
    
FROM COMPLIANCE_DEMO.ML.LLM_ANALYSIS
""").collect()

print("✅ Created ensemble results view")

# Show priority breakdown
print("\n--- Review Priority Distribution ---")
session.sql("""
    SELECT 
        REVIEW_PRIORITY,
        COUNT(*) as COUNT
    FROM COMPLIANCE_DEMO.ML.ENSEMBLE_RESULTS
    GROUP BY 1
    ORDER BY 
        CASE REVIEW_PRIORITY
            WHEN 'CRITICAL' THEN 1
            WHEN 'HIGH' THEN 2
            WHEN 'MEDIUM' THEN 3
            ELSE 4
        END
""").show()

## 5. Cost Analysis

Compare the cost of different approaches.

In [ ]:
# Cost comparison (estimated)
total_emails = 10000
ml_flagged = flagged_count

# Estimated costs per email (approximate)
ml_cost_per_email = 0.0001  # ~$0.0001 per email for XGBoost inference
llm_cost_per_email = 0.01   # ~$0.01 per email for Claude claude-opus-4-5

# Calculate costs for different approaches
all_llm_cost = total_emails * llm_cost_per_email
all_ml_cost = total_emails * ml_cost_per_email
tiered_cost = (total_emails * ml_cost_per_email) + (ml_flagged * llm_cost_per_email)

print("=" * 60)
print("COST ANALYSIS: 10,000 EMAILS")
print("=" * 60)
print(f"""
┌────────────────────────────────────────────────────────────┐
│  APPROACH 1: All LLM (Claude claude-opus-4-5 on every email)        │
│  └── Cost: ${all_llm_cost:,.2f}                                    │
│  └── LLM calls: {total_emails:,}                                 │
├────────────────────────────────────────────────────────────┤
│  APPROACH 2: All ML (XGBoost only)                         │
│  └── Cost: ${all_ml_cost:,.2f}                                     │
│  └── Limitation: May miss nuanced violations               │
├────────────────────────────────────────────────────────────┤
│  APPROACH 3: TIERED (ML filter → LLM on flagged)           │
│  └── Cost: ${tiered_cost:,.2f}                                     │
│  └── ML scans: {total_emails:,}, LLM deep-dives: {ml_flagged:,}      │
│  └── SAVINGS vs All-LLM: {(1 - tiered_cost/all_llm_cost)*100:.0f}%                          │
└────────────────────────────────────────────────────────────┘
""")

## Summary

**What we demonstrated:**
- ML as a fast, cheap filter (classifies all 10K emails in seconds)
- Claude claude-opus-4-5 for deep analysis (only on ML-flagged emails)
- Ensemble approach combining both for maximum accuracy
- Priority scoring for human review queue

**Key Benefits of Tiered Architecture:**
1. **Cost Efficiency:** 80-90% reduction vs all-LLM
2. **Speed:** Instant ML classification, targeted LLM analysis
3. **Accuracy:** ML catches obvious cases, LLM handles nuance
4. **Explainability:** Claude provides reasoning for each decision

**Next:** In notebook 05, we'll fine-tune an LLM specifically for compliance classification.